In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Following along the pcg_skel docs and tutorial:
## https://caveconnectome.github.io/pcg_skel/
## https://caveconnectome.github.io/pcg_skel/tutorial/

In [4]:
import sys
sorted(sys.path)

['',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip',
 '/Users/keith.wiley/virtual_envs/work_env3/lib/python3.9/site-packages']

In [5]:
import pcg_skel
pcg_skel.__file__

In [6]:
print(pcg_skel.__path__)

_NamespacePath(['/Users/keith.wiley/Work/Code/pcg_skel'])


In [7]:
pcg_skel.pcg_skeleton

AttributeError: module 'pcg_skel' has no attribute 'pcg_skeleton'

In [1]:
%%time
import caveclient
import pcg_skel

datastack = 'minnie65_public'
client = caveclient.CAVEclient(datastack)
client.materialize.version = 795 # Ensure we will always use this data release
root_id = 864691135397503777
skel = pcg_skel.pcg_skeleton(root_id=root_id, client=client)

CPU times: user 889 ms, sys: 2.59 s, total: 3.48 s
Wall time: 2.35 s


In [6]:
skel.path_length()

15474161.0

In [8]:
%%time
import caveclient
from pcg_skel import pcg_skel

datastack = 'minnie65_public'
client = caveclient.CAVEclient(datastack)
client.materialize.version = 795 # Ensure we will always use this data release

root_id = 864691135397503777

# Get the location of the soma from nucleus detection:
root_resolution = [1,1,1] # Could be another resolution as well, but this will mean the location is in nm.
soma_df = client.materialize.views.nucleus_detection_lookup_v1(
    pt_root_id = root_id
    ).query(
        desired_resolution = root_resolution
    )
soma_location = soma_df['pt_position'].values[0]

# Use the above parameters in the skeletonization:

skel = pcg_skel.coord_space_skeleton(
    root_id,
    client,
    root_point=soma_location,
    root_point_resolution=root_resolution,
    collapse_soma=True,
    collapse_radius=7500,
)

The `client.materialize.views` interface is experimental and might experience breaking changes before the feature is stabilized.


CPU times: user 1.02 s, sys: 83.1 ms, total: 1.1 s
Wall time: 15.5 s


In [9]:
%%time
nrn = pcg_skel.pcg_meshwork(
    root_id = root_id,
    client = client,
    root_point = soma_location,
    root_point_resolution = root_resolution,
    collapse_soma = True,
    collapse_radius = 7500,
    synapses=True,
)

CPU times: user 964 ms, sys: 59.2 ms, total: 1.02 s
Wall time: 30.1 s


In [15]:
nrn.anno.pre_syn.df.head()

,id,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,pre_pt_position,post_pt_position,ctr_pt_position,pre_pt_level2_id,pre_pt_mesh_ind,pre_pt_mesh_ind_filt
0,222458266,19216,94159084433346408,864691135397503777,94159084433355759,"[854472.0, 414056.0, 915040.0]","[854640.0, 413680.0, 915440.0]","[854592.0, 413984.0, 915360.0]",166216678470845194,12662,12662
1,415060597,4352,109934464683069477,864691135397503777,109934533402545858,"[1313904.0, 795320.0, 821960.0]","[1313808.0, 795720.0, 822000.0]","[1313720.0, 795432.0, 822040.0]",181992058720879676,14651,14651
2,234975568,1688,95282373134387093,864691135397503777,95282373134380129,"[887920.0, 336216.0, 960480.0]","[888256.0, 336392.0, 960440.0]","[888120.0, 336488.0, 960480.0]",167339967171790083,13138,13138
3,385655265,25748,107963178377293582,864691135397503777,107892809633163367,"[1255864.0, 765688.0, 1003600.0]","[1255544.0, 765800.0, 1003920.0]","[1255768.0, 765800.0, 1003960.0]",180020772415209494,14572,14572
4,255404116,388,96843954657937028,864691135397503777,96843954657942059,"[932720.0, 736840.0, 1000320.0]","[932736.0, 737032.0, 1000560.0]","[932784.0, 736928.0, 1000360.0]",168901548695356210,13452,13452


In [16]:
nrn.anno.post_syn.df.head()

,id,size,pre_pt_supervoxel_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position,post_pt_level2_id,post_pt_mesh_ind,post_pt_mesh_ind_filt
0,172461633,15200,90359378338883207,90359378338887551,864691135397503777,"[745048.0, 418840.0, 888160.0]","[744744.0, 419112.0, 888240.0]","[744776.0, 419024.0, 888160.0]",162416972376572296,7995,7995
1,142677506,22532,88108334439511704,88108265720062299,864691135397503777,"[678880.0, 441128.0, 890840.0]","[679104.0, 440728.0, 890800.0]","[679216.0, 440768.0, 890720.0]",160165859757654524,3412,3412
2,145066644,4532,88036110403862351,88036110403865679,864691135397503777,"[677880.0, 387048.0, 939680.0]","[677400.0, 387152.0, 939720.0]","[677648.0, 387344.0, 939760.0]",160093704441299965,3236,3236
3,149896539,2660,88811815655855809,88882184400027792,864691135397503777,"[700472.0, 436192.0, 873960.0]","[700576.0, 436056.0, 874120.0]","[700536.0, 436168.0, 874080.0]",160939778437546796,4447,4447
4,170547369,2320,90357866644502688,90357866644495463,864691135397503777,"[743560.0, 374824.0, 925320.0]","[743824.0, 374696.0, 924840.0]","[743512.0, 374784.0, 925040.0]",162415460682301674,7940,7940
